What is Image Augmentation 

Image Augmentation is the process of taking images that are already in a training dataset and manipulating them to create many altered versions of the same image. This both provides more images to train on, and can also help expose our classifier to a wider variety of lighting and coloring situations so as to make our classifier more robust.

What is the need of Image Augmentation ?

To combat the high expense of collecting thousands of training images, image augmentation has been developed in order to generate training data from an existing dataset.

Importing the libraries and data

In [26]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [27]:
batch_size = 32
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

Splitting the data into training and test samples

In [28]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Convert class vectors to binary class metrices

In [29]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


Defining the CNN Architecture

In [30]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [31]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


Image Preprocessing and Data Augmentation

Running the Default Base Model

Creating an Image using ImageDataGenerator() 

To Augment our image data with keras, we simply call the ImageDataGenerator() function and pass a list of parameters describing the alterations that we want it to perform on the images. We will then call the fit() function on our image generator which will apply the changes to the images batch by batch. By default, the modifications will be applied randomly, so not every image will be changed every time. 

In [32]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None)
        # fraction of images reserved for validation (strictly between 0 and 1)
        #validation_split=0.0)

In [33]:
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch= 1000, epochs= 5)

Epoch 1/5
1000/1000 [==============================] - 246s - loss: 1.9327 - acc: 0.2864   
Epoch 2/5
1000/1000 [==============================] - 244s - loss: 1.6770 - acc: 0.3854   
Epoch 3/5
1000/1000 [==============================] - 246s - loss: 1.5541 - acc: 0.4352   
Epoch 4/5
1000/1000 [==============================] - 250s - loss: 1.4762 - acc: 0.4624   
Epoch 5/5
1000/1000 [==============================] - 241s - loss: 1.4113 - acc: 0.4935   


Evaluating the model on test data

In [34]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /Users/neelambabel/Desktop/Cognitive Computing/saved_models/keras_cifar10_trained_model.h5 
 9984/10000 [============================>.] - ETA: 0sTest loss: 1.25454781742
Test accuracy: 0.555


Case 1 : Rotating the images by 60 Degrees

In [35]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=60,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.4,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None)
        # fraction of images reserved for validation (strictly between 0 and 1)
        #validation_split=0.0)

In [36]:
# Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch= 1000, epochs= 5)

Epoch 1/5
1000/1000 [==============================] - 267s - loss: 1.7226 - acc: 0.3831   
Epoch 2/5
1000/1000 [==============================] - 251s - loss: 1.6680 - acc: 0.4000   
Epoch 3/5
1000/1000 [==============================] - 237s - loss: 1.6348 - acc: 0.4108   
Epoch 4/5
1000/1000 [==============================] - 373s - loss: 1.6059 - acc: 0.4230   
Epoch 5/5
1000/1000 [==============================] - 238s - loss: 1.5757 - acc: 0.4326   


In [37]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /Users/neelambabel/Desktop/Cognitive Computing/saved_models/keras_cifar10_trained_model.h5 
 9984/10000 [============================>.] - ETA: 0sTest loss: 1.26494939251
Test accuracy: 0.5553


Case 2 : Shifting the images Horizontally by 40%

In [44]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.4,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None)
        # fraction of images reserved for validation (strictly between 0 and 1)
        #validation_split=0.0)

In [39]:
 # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch= 1000, epochs= 5)

Epoch 1/5
1000/1000 [==============================] - 231s - loss: 1.4095 - acc: 0.4957   
Epoch 2/5
1000/1000 [==============================] - 227s - loss: 1.3607 - acc: 0.5095   
Epoch 3/5
1000/1000 [==============================] - 231s - loss: 1.3330 - acc: 0.5204   
Epoch 4/5
1000/1000 [==============================] - 236s - loss: 1.2932 - acc: 0.5356   
Epoch 5/5
1000/1000 [==============================] - 233s - loss: 1.2837 - acc: 0.5457   


In [40]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /Users/neelambabel/Desktop/Cognitive Computing/saved_models/keras_cifar10_trained_model.h5 
 9984/10000 [============================>.] - ETA: 0sTest loss: 1.08292009888
Test accuracy: 0.6131


Case 3 - Flip Images Vertically

In [41]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=False,  # randomly flip images
        vertical_flip=True,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None)
        # fraction of images reserved for validation (strictly between 0 and 1)
        #validation_split=0.0)

In [42]:
# Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch= 1000, epochs= 5)

Epoch 1/5
1000/1000 [==============================] - 230s - loss: 1.4211 - acc: 0.4828   
Epoch 2/5
1000/1000 [==============================] - 252s - loss: 1.3391 - acc: 0.5120   
Epoch 3/5
1000/1000 [==============================] - 238s - loss: 1.3089 - acc: 0.5304   
Epoch 4/5
1000/1000 [==============================] - 238s - loss: 1.2673 - acc: 0.5453   
Epoch 5/5
1000/1000 [==============================] - 250s - loss: 1.2485 - acc: 0.5497   


In [43]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /Users/neelambabel/Desktop/Cognitive Computing/saved_models/keras_cifar10_trained_model.h5 
 9984/10000 [============================>.] - ETA: 0sTest loss: 1.06653157883
Test accuracy: 0.6196
